# Connect to ElasticSearch 

In [1]:
from pprint import pprint 
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200", 
    basic_auth=("elastic", "6AqhOxi*CPXYvCZl7Iln"), 
    verify_certs=False)
client_info = es.info() 
print("Connected to Elasticsearch!")
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'mIJwhjTmStW54eKFEwQnMA',
 'name': 'f12c85f397e4',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2026-01-29T10:05:46.708397977Z',
             'build_flavor': 'default',
             'build_hash': '17b451d8979a29e31935fe1eb901310350b30e62',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.3.0'}}


c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\elasticsearch\_sync\client\__init__.py:313: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Inserting Documents 
Because we are using the dense_vector data type, we need to do the mapping manually 

In [3]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(
    index="my_index", 
    mappings={
        "properties": {
            "sides_length": {
                "type": "dense_vector", 
                "dims": 4
            }, 
            "shape": {
                "type": "keyword"
            }
        }
    }, 
)

c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

# Valid Case
We specifies that sides_length should be a vector with 4 values. Since this condition was met the operation was executed successfully

In [4]:
from pprint import pprint

repsonse = es.index(
    index='my_index', 
    id=1, 
    document={
        "shape": "square", 
        "sides_length": [5, 5, 5, 5], 
    }
)

pprint(repsonse.body)

c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'_id': '1',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


let's check the mapping. We can see that sides_length is a dense_vector with 4 dimensions and cosine set as the similarity measure. These are parameters we can modify if needed. Since we didn't specify them when manually setting the mapping. Elasticsearch automatically applied the default values. 

In [5]:
pprint(es.indices.get_mapping(index='my_index').body)

{'my_index': {'mappings': {'properties': {'shape': {'type': 'keyword'},
                                          'sides_length': {'dims': 4,
                                                           'index': True,
                                                           'index_options': {'ef_construction': 100,
                                                                             'm': 16,
                                                                             'type': 'int8_hnsw'},
                                                           'similarity': 'cosine',
                                                           'type': 'dense_vector'}}}}}


c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Invalid Case 
in this case, the operation fails because indexing a matrix is not supported with dense_vector field type. 

In [6]:
response = es.index(
    index='my_index', 
    id=2, 
    document={
        'shape': "square", 
        "sides_length": [[5, 5], [5, 5]],
    }
)

print(response.body)

c:\Users\Asus\Desktop\sastokinmel\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


BadRequestError: BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [START_ARRAY]', Failed to parse object: expecting token of type [VALUE_NUMBER] but found [START_ARRAY])